# Modification de la donnée brute

L\'objectif est de garder trace des informations d\'origine du fichier si les données sont manipulées par la suite (agrégées, modifiées, déplacées ...).

Ce notebook utilise les fichiers présents dans le s3 dans [00_RawData](https://s3.console.aws.amazon.com/s3/buckets/calfdata/INPI/TC_1/00_RawData/?region=eu-west-3).

Pour chaque fichier présent dans le dossier source:

- Il déduit à partir du nom du fichier les différentes variables de classification grâce à la fonction `̀ get_file_infos`̀ .
        - ``origin_type`` : Flux ou Stock.
        - ``origin`` : Initial ou Partiel pour Stock, NEW ou EVT pour Flux.
        - ``nature`` : ACTES, ETS, OBS, COMPTES_ANNUELS, PM, PP, REP.

- Il calcule le dossier destination du fichier csv et crée une liste de fichiers à envoyer.

Grâce à la fonction `̀ prepare_object`̀ :
- Il lit un fichier les colonnes suivantes aux csv:
- Il ajoute les colonnes suivantes au csv:

    - ``csv_source`` : nom du fichier source. Exemple format: 3801_189_20180130_065752_9_ets_nouveau_modifie_EVT.csv
    - ``file_timestamp`` = la date du fichier ou une date du fichier source (par ex date_greffe) pour les stock initial qui ont tous par défaut une date de fichier à 2017-05-24.
    - ``origin_type`` : Flux ou Stock.
    - ``origin`` : Initial ou Partiel pour Stock, NEW ou EVT pour Flux.
    - ``nature`` : ACTES, ETS, OBS, COMPTES_ANNUELS, PM, PP, REP.
        
- il écrit directement dans S3 le fichier préparé.

La donnée est stockée dans le dossier [01_donnee_source](https://s3.console.aws.amazon.com/s3/buckets/calfdata/INPI/TC_1/01_donnee_source/?region=eu-west-3) et l'arborescence simplifiée.

La fonction `̀ log_init`̀  s'occupe de gérer les logs pour faciliter la reprise sur erreur.

Options:
- Il est possible de définir des filtres pour ne prendre qu'une certaine nature de fichiers à l'intérieur d'un dossier.
- Il est possible de faire retourner le script sur la liste des erreurs ou sur une liste libre de clés.

In [ ]:
from tqdm import tqdm

In [ ]:
import utils as u

In [ ]:
def get_file_infos(filename,dl_path):
    
    """
    Computes information that are in the filename to classify its content.
    cf INPI files documentation here : https://coda.io/d/CreditAgricole_dCtnoqIftTn/INSEE-INPI_suTDp

    args:
         - filename: Any filename
         ex : '0101_S1_20170504_11_obs.csv'
         - dl_path : local path of file

    returns:
        A list of informations : (nature_,origin_,suborigin_,year_,timestamp_).

    """
    
    f_s=dl_path.split('/')
    suborigin_=f_s[6]
    origin_=f_s[2]
    nature_=f_s[5]
    year_=f_s[3]
    
    import datetime
    from datetime import datetime

    f_s = filename.split('_')
    date_=f_s[2]
    year_=date_[0:4]
    month_=date_[4:6]
    day_=date_[6:9]
    time_= f_s[3]

    datetime_str=date_ + '_' + time_
    timestamp_=datetime.strptime(datetime_str,"%Y%m%d_%H%M%S")
    
    return (nature_,origin_,suborigin_,year_,timestamp_)

# Connect to S3

In [ ]:
# Import S3 connectors librairies

In [ ]:
!pip install git+git://github.com/thomaspernet/aws-python

In [ ]:
!pip install --upgrade git+git://github.com/thomaspernet/aws-python

In [ ]:
from awsPy.aws_authorization import aws_connector
from awsPy.aws_s3 import service_s3
from awsPy.aws_athena import service_athena

In [ ]:
# Connect to S3
import os
from pathlib import Path

bucket = 'calfdata'
path = os.getcwd()
parent_path = str(Path(path).parent)
path_cred = "{}/programme_matching/credential_AWS.json".format(parent_path)

con = aws_connector.aws_instantiate(credential = path_cred, region = 'eu-west-3')
client= con.client_boto()
s3 = service_s3.connect_S3(client = client, bucket = 'calfdata')

bucket_name='calfdata'
bucket = client['resource'].Bucket(bucket_name)    

# From Raw Folder

In [ ]:
import pandas as pd
import io
from io import StringIO
import time

In [ ]:
# Initiate log files
def log_init():
    """
    Initiate log files for viewed, treated and files in error.

    """

    logv_name="{}/{}_{}-{}".format('data',timestr,folder_str,'viewed.csv')
    logv = open(logv_name, 'w')
    print('key', file=logv)
    logt_name="{}/{}_{}-{}".format('data',timestr,folder_str,'treated.csv')
    logt = open(logt_name, 'w')
    print('key', file=logt)
    loge_name="{}/{}_{}-{}".format('data',timestr,folder_str,'errors.csv')
    loge = open(loge_name, 'w')
    print('key;error', file=loge)    
    
    return (logtd,logv,logt,loge)

In [ ]:
def prepare_object(obj):
    """
    Prepares a Raw csv file from INPI :
        - Read file
        - Add information in columns
        - Write result directly in S3

    Arg:
        - obj : an S3 object
        
    Return:
        - the destination path in S3 of the prepared file generated

    """
    
    key = obj.key
    body = obj.get()['Body'].read()
    s=key.split('/')
    filename=s[len(s)-1]

    (nature_,origin_,suborigin_,year_,timestamp_) = u.get_file_infos(filename)
    dest_path="{}/{}/{}/{}/{}".format('INPI/TC_1/01_donnee_source',origin_,year_,nature_,suborigin_)
    dest_full_path="{}/{}".format(dest_path,filename)
            
    df = pd.read_csv(io.BytesIO(body), header=0, dtype=str, sep = ';',error_bad_lines=False)
    if nature_ == 'REP':
        # Rename col 'date_greffe' to 'Date_Greffe'
        df=df.rename(columns={"date_greffe": "Date_Greffe"})

    df['csv_source']=filename
    df['nature']=nature_
    df['type']=origin_
    df['origin']=suborigin_

    if origin_=='Stock' and year_ == '2017' and nature_ == 'ACTES': 
        # Tout le stock étant initialisé à la même date en 2017, on utilise plutôt la Date_Dépôt
        df['file_timestamp']= df['Date_Dépôt'].apply(lambda x : datetime.strptime(x,"%Y-%m-%d"))
    elif origin_=='Stock' and year_ == '2017' and nature_ == 'COMPTES_ANNUELS': 
        # Tout le stock étant initialisé à la même date en 2017, on utilise plutôt la Date_Dépôt
        df['file_timestamp']= df['Date_Dépôt'].apply(lambda x : datetime.strptime(x,"%Y-%m-%d"))
    elif origin_=='Stock' and year_ == '2017': 
        # Tout le stock étant initialisé à la même date en 2017, on utilise plutôt la Date_Greffe
        df['file_timestamp']= df['Date_Greffe'].apply(lambda x : datetime.strptime(x,"%Y-%m-%d"))
    else:
        df['file_timestamp']=timestamp_    

    # Save file to destination in S3
    # Create buffer
    csv_buffer = StringIO()
    # Write dataframe to buffer
    df.to_csv(csv_buffer, sep=";", index=False)
    # Create S3 object
    s3_resource = client['resource']
    # Write buffer to S3 object
    s3_resource.Object(bucket_name, dest_full_path).put(Body=csv_buffer.getvalue())

    
    return dest_full_path

# Prepare all items in a folder

In [ ]:
# Select folder
folder='INPI/TC_1/00_RawData/public/IMR_Donnees_Saisies/tc/flux/2018/01/01'
folder_str='rawFlux20180101'

In [ ]:
# List all objects in S3 folder
list_bucket = bucket.objects.filter(Prefix=folder)

In [ ]:
# Optional filter on certain filenames

filter_ = (
#'1_PM.csv',
#'2_PM_EVT.csv',
#'3_PP.csv',
#'4_PP_EVT.csv',
#'5_rep.csv',
#'6_rep_nouveau_modifie_EVT.csv',
#'7_rep_partant_EVT.csv',
#'8_ets.csv',
#'9_ets_nouveau_modifie_EVT.csv',
'10_ets_supprime_EVT.csv',
#'11_obs.csv',
'12_actes.csv',
'13_comptes_annuels.csv'
    )

list_bucket_filter=[]
for obj in tqdm(list_bucket):
    if obj.key.endswith(filter_):
        list_bucket_filter.append(obj)
        
list_bucket=list_bucket_filter if filter_ else list_bucket_filter

In [ ]:
# Save todo list to a file
import csv
timestr = time.strftime("%Y%m%d-%H%M%S")
logtd_name="{}/{}_{}-{}".format('data',timestr,folder_str,'todo.csv')
logtd = open(logtd_name, 'w')
with logtd as myfile:
    wr = csv.writer(myfile,delimiter=';',quoting=csv.QUOTE_ALL)
    wr.writerow(['key'])
    for obj in tqdm(list_bucket):
        wr.writerow([obj.key])
logtd.close()

In [ ]:
(logtd,logv,logt,loge)=log_init()

In [ ]:
for obj in tqdm(list_bucket):

    try:
        print(obj.key, file=logv)
        prepare_object(obj)
        print(obj.key, file=logt)
        
    except Exception as e:
        print((obj.key,e), file=loge)

logv.close()
logt.close()
loge.close()

# Stats and error recovery

In [ ]:
# Stats 
dftd=pd.read_csv(logtd.name,header=0, sep=';')# Todo
td=len(dftd)
dft=pd.read_csv(logt.name,header=0, sep=';')
t=len(dft)
dfe=pd.read_csv(loge.name,header=0, sep=';',usecols = ['key'])
e=len(dfe)
# Find untreated objects
dfnt=pd.DataFrame()
dfnt=dftd.merge(dft,indicator = True, how='left').loc[lambda x : x['_merge']!='both']
dfnt=dfnt.drop(columns=['_merge'])
nt=len(dfnt)
# Add errors
dfnt=dfnt.append(dfe,sort=True)
nt=len(dfnt)



# Print stats
print('Folder :' + folder)
if filter:
    print("Filter : %s" % (filter_,))
print('To do : ' + str(td))
print('Treated : ' + str(t))
print('Errors : ' + str(e))
print('Retake : ' + str(nt))

# Prepare from a file list

In [ ]:
# Save errors to a new todo
list_bucket_keys = dfnt['key'].values.tolist()

In [ ]:
# Or Create the new todo from a list of Missing files
folder_str='prepFluxMissing2018'
df=pd.read_csv('data/count/missingPrepFlux2018.csv',header=None, sep=';')# Todo
df.columns = ['key']
list_bucket_keys = df['key'].values.tolist()

In [ ]:
# Save list to a new todo
import csv
timestr = time.strftime("%Y%m%d-%H%M%S")
logtd_name="{}/{}_{}-{}".format('data',timestr,folder_str,'todo.csv')
logtd = open(logtd_name, 'w')
with logtd as myfile:
    wr = csv.writer(myfile,delimiter=';',quoting=csv.QUOTE_ALL)
    wr.writerow(['key'])
    for key in tqdm(list_bucket_keys):
        wr.writerow([key])
logtd.close()

In [ ]:
(logtd,logv,logt,loge)=log_init()

In [ ]:
# Restart on errors
for n,key in tqdm(enumerate(list_bucket_keys)):
    try:
        print(key, file=logv)
        
        # get object in S3 by its key
        obj=client['resource'].Object(bucket_name=bucket_name, key=key)
        f=prepare_object(obj)
        print(key, file=logt)
        
    except Exception as e:
        print((key,e), file=loge)

logv.close()
logt.close()
loge.close()